In [1]:
import pandas as pd 
import numpy as np

import BigWindowHAI
data = pd.read_csv("normalized_HAI.csv")
splits = BigWindowHAI.get_data(data = data, n_splits=10)


In [ ]:
import numpy as np
#now let us check the similarity between these datasets. 
def get_metrics(index_of_split):   
    stats_df3 = pd.DataFrame({
        'mean': splits[index_of_split].mean(),      # Mean of each column
        'std': splits[index_of_split].std(),        # Std of each column
        'median': splits[index_of_split].median(),  # Median of each column
        'min': splits[index_of_split].min(),        # Minimum value
        'max': splits[index_of_split].max(),        # Maximum value
        'range': splits[index_of_split].max() - splits[index_of_split].min(),  # Range
        'iqr': splits[index_of_split].quantile(0.75) - splits[index_of_split].quantile(0.25),  # Interquartile Range
        'skewness': splits[index_of_split].skew(),  # Skewness of each column
        'kurtosis': splits[index_of_split].kurt(),  # Kurtosis of each column
    })
    return stats_df3

def compare_metrics(stats1, stats2, method='euclidean'):
    # Ensure stats1 and stats2 are aligned
    stats1, stats2 = stats1.align(stats2, join='inner', axis=0)
    
    # Calculate the differences
    diff = stats1.values - stats2.values
    
    if method == 'euclidean':
        # Euclidean distance
        score = np.sqrt(np.sum(diff ** 2))
    elif method == 'manhattan':
        # Manhattan distance (sum of absolute differences)
        score = np.sum(np.abs(diff))
    elif method == 'cosine':
        # Cosine similarity (1 - cosine distance)
        dot_product = np.sum(stats1.values * stats2.values)
        norm1 = np.sqrt(np.sum(stats1.values ** 2))
        norm2 = np.sqrt(np.sum(stats2.values ** 2))
        score = 1 - dot_product / (norm1 * norm2)
    else:
        raise ValueError("Invalid method. Choose 'euclidean', 'manhattan', or 'cosine'.")
    
    return score


def check_similarity(split1index, split2index, method= 'euclidean'):
    return compare_metrics(get_metrics(split1index) , get_metrics(split2index), method=method)




#here we check how similar our data splits to each other
positive_infinity = float('inf')

for i in range (1,10):
    min= float('inf')
    for j in range(0, i):
        if (check_similarity(i, j) < min):
            min = check_similarity(i,j)
            most_similar = j
    print(f'for split {i} : ')
    print(f'is most similar with {most_similar} is {min}') 

for split 1 : 
is most similar with 0 is 64.96496229525859
for split 2 : 
is most similar with 1 is 62.182200417203795
for split 3 : 
is most similar with 1 is 46.915163806580374
for split 4 : 
is most similar with 2 is 322.7665954404913
for split 5 : 
is most similar with 1 is 84.09045956592809
for split 6 : 
is most similar with 2 is 46.80315582741101
for split 7 : 
is most similar with 3 is 37.62224381684561
for split 8 : 
is most similar with 7 is 28.776663150742944
for split 9 : 
is most similar with 1 is 24.064076578967846


In [6]:
import numpy as np
import pandas as pd

def get_metrics(index_of_split):   
    return pd.DataFrame({
        'mean': splits[index_of_split].mean(),
        'std': splits[index_of_split].std(),
        'median': splits[index_of_split].median(),
        'min': splits[index_of_split].min(),
        'max': splits[index_of_split].max(),
        'range': splits[index_of_split].max() - splits[index_of_split].min(),
        'iqr': splits[index_of_split].quantile(0.75) - splits[index_of_split].quantile(0.25),
        'skewness': splits[index_of_split].skew(),
        'kurtosis': splits[index_of_split].kurt(),
    })

def compare_metrics(stats1, stats2):
    stats1, stats2 = stats1.align(stats2, join='inner', axis=0)
    diff = stats1.values - stats2.values
    return np.sqrt(np.sum(diff ** 2))  # Euclidean distance

# Compute similarity scores between all split pairs
similarity_scores = []
for i in range(1, 10):
    for j in range(i):
        score = compare_metrics(get_metrics(i), get_metrics(j))
        similarity_scores.append(score)

# **Determine the threshold dynamically**
SIMILARITY_THRESHOLD = np.percentile(similarity_scores, 10)  # Skip top 10% most similar splits
print(f"🔹 Auto-Determined Similarity Threshold: {SIMILARITY_THRESHOLD:.5f}")

# **Now, filter splits based on this threshold**
selected_splits = [0]  # Always keep first split

for i in range(1, 10):
    min_similarity = float('inf')
    most_similar = None

    for j in selected_splits:
        similarity_score = compare_metrics(get_metrics(i), get_metrics(j))

        if similarity_score < min_similarity:
            min_similarity = similarity_score
            most_similar = j

    print(f"Split {i} is most similar to Split {most_similar} with score: {min_similarity:.5f}")

    # **Decide: Skip or Keep**
    if min_similarity >= SIMILARITY_THRESHOLD:
        selected_splits.append(i)  # Keep the split
        print(f"✅ Keeping Split {i}")
    else:
        print(f"❌ Skipping Split {i} (Too Similar)")

print("\nFinal Selected Splits:", selected_splits)


🔹 Auto-Determined Similarity Threshold: 39.22531
Split 1 is most similar to Split 0 with score: 64.96496
✅ Keeping Split 1
Split 2 is most similar to Split 1 with score: 62.18220
✅ Keeping Split 2
Split 3 is most similar to Split 1 with score: 46.91516
✅ Keeping Split 3
Split 4 is most similar to Split 2 with score: 322.76660
✅ Keeping Split 4
Split 5 is most similar to Split 1 with score: 84.09046
✅ Keeping Split 5
Split 6 is most similar to Split 2 with score: 46.80316
✅ Keeping Split 6
Split 7 is most similar to Split 3 with score: 37.62224
❌ Skipping Split 7 (Too Similar)
Split 8 is most similar to Split 6 with score: 32.28200
❌ Skipping Split 8 (Too Similar)
Split 9 is most similar to Split 1 with score: 24.06408
❌ Skipping Split 9 (Too Similar)

Final Selected Splits: [0, 1, 2, 3, 4, 5, 6]
